In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# Sklearn
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, PowerTransformer,QuantileTransformer, OneHotEncoder

#Visualization
import matplotlib.cm as cm
from sklearn import random_projection

import datetime

In [4]:
dtype_dict = {
    'ProductFamily_ID':'category',
    'ProductCategory_ID':'category',
    'ProductBrand_ID':'category',
    'ProductName_ID':'category',
    'ProductPackSKU_ID':'category',
    'Point-of-Sale_ID':'category',
    'Measures':'category',
    'Quantity':'float32',
    'Total_Sales':'float32',
    'Value':'float32',
    'Week': 'int32',
    'Year':'int32'
}
original_path = r'D:\NOVAIMS_MAA\NOVAIMS_MAA_2020e21_BusinessCasesDataScience_MindOverData_RetailChallenge.csv'
path = r'D:\NOVAIMS_MAA\NOVAIMS_MAA_final_1.csv'

In [3]:
example = pd.read_csv(path,dtype=dtype_dict,parse_dates=['Date'],nrows = 1000,usecols = [1,2,3,4,5,6,7,8,9])
example.head()

,ProductFamily_ID,ProductCategory_ID,ProductBrand_ID,ProductName_ID,ProductPackSKU_ID,Point-of-Sale_ID,Date,Quantity,Total_Sales
0,16,11,306,649,1970,1,2017-03-04,2.0,1540.0
1,16,11,306,649,1970,1,2016-05-02,4.0,3080.0
2,16,11,306,649,1970,1,2016-10-24,2.0,1540.0
3,16,11,306,649,1970,1,2017-10-13,2.0,1620.0
4,16,11,306,649,1970,1,2017-10-14,2.0,1620.0


In [4]:
df = pd.read_csv(path,dtype=dtype_dict,usecols = [4,7,8])

In [5]:
df.Date  = df.Date.astype("datetime64")

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91187504 entries, 0 to 91187503
Data columns (total 3 columns):
 #   Column          Dtype         
---  ------          -----         
 0   ProductName_ID  category      
 1   Date            datetime64[ns]
 2   Quantity        float32       
dtypes: category(1), datetime64[ns](1), float32(1)
memory usage: 1.2 GB


In [7]:
df['Week'] = df['Date'].dt.week

In [8]:
df['Year'] = df['Date'].dt.year

In [9]:
df = df[df['Year'] >= 2018]

In [10]:
df.drop(columns = "Date",inplace = True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45131103 entries, 27 to 91187495
Data columns (total 4 columns):
 #   Column          Dtype   
---  ------          -----   
 0   ProductName_ID  category
 1   Quantity        float32 
 2   Week            int64   
 3   Year            int64   
dtypes: category(1), float32(1), int64(2)
memory usage: 1.3 GB


In [12]:
#df = df.groupby(['Year','Week','ProductName_ID']).sum()['Quantity'].reset_index()
#df.to_csv(r'D:\NOVAIMS_MAA\timeseries\product_week_year.csv')

In [5]:
df = pd.read_csv(r'D:\NOVAIMS_MAA\timeseries\product_week_year.csv',dtype=dtype_dict,usecols = [1,2,3,4])

In [8]:
agg_df = df.groupby(['Year','Week','ProductName_ID']).sum()['Quantity'].reset_index()

In [10]:
agg_df.dropna(inplace=True)

In [12]:
agg_df.to_csv(r'D:\NOVAIMS_MAA\timeseries\agg_product_week_year.csv')